In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import re
import matplotlib.pyplot as plt

In [2]:
## Upload RCPier Repor
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Bent 04-19 Loadings.txt to Bent 04-19 Loadings.txt
User uploaded file "Bent 04-19 Loadings.txt" with length 267478 bytes


In [3]:
fn

'Bent 04-19 Loadings.txt'

In [4]:
## Convert the LPILE txt report to strings
import chardet
with open(fn, 'rb') as f:
  result = chardet.detect(f.read())
  xencode = result['encoding']
print(f"Detected encoding: {xencode}")

with open(fn, 'r', encoding=xencode) as f:
  textog = f.read()
textog

Detected encoding: ascii


'\x0c\n \n \nLOADS\n=====\n \nPier View   : Downstation.\n   Load Cases: 124\n   Longitudinal Reaction: Simple Span Distribution\n   Selected Vehicles:                      \n \n            Load: Two Design Trucks + Lane Load   \n \nTranverse Positioning\n=====================\n \nNumber of loaded lanes = All combinations\nLive Load Positions = Constant Spacing\nMinimum Distance from Curb = 2.000000\nCenter to Center Spacing = 10.000000\n \n \nGenerate Braking/Longitudinal Force = Selected\nGenerate Centrifugal Force = Not Selected\n \n \nNumber of single regular truck positions = 28\n Number of all possible combinations of regular truck positions = 30888\n \n \n      Loadcase ID: DC1     Name: \n      Multiplier =  1.000\n \n      Bearing loads:\n         Line #    Bearing #   Dir.    Load,  kips\n         -------------------------------------------------\n           1          1         Y      -141.95\n           1          2         Y      -149.98\n           1          3         Y 

In [5]:
## Define the begin and end pattern for the re.

startp = "\n         -------------------------------------------------\n"
endp = "\n \n      Auto generation details"
textog = textog[:textog.find("Selected load groups")]
textog

'\x0c\n \n \nLOADS\n=====\n \nPier View   : Downstation.\n   Load Cases: 124\n   Longitudinal Reaction: Simple Span Distribution\n   Selected Vehicles:                      \n \n            Load: Two Design Trucks + Lane Load   \n \nTranverse Positioning\n=====================\n \nNumber of loaded lanes = All combinations\nLive Load Positions = Constant Spacing\nMinimum Distance from Curb = 2.000000\nCenter to Center Spacing = 10.000000\n \n \nGenerate Braking/Longitudinal Force = Selected\nGenerate Centrifugal Force = Not Selected\n \n \nNumber of single regular truck positions = 28\n Number of all possible combinations of regular truck positions = 30888\n \n \n      Loadcase ID: DC1     Name: \n      Multiplier =  1.000\n \n      Bearing loads:\n         Line #    Bearing #   Dir.    Load,  kips\n         -------------------------------------------------\n           1          1         Y      -141.95\n           1          2         Y      -149.98\n           1          3         Y 

In [6]:
## Function
## Convert load case to dataframe
def convSPtoDF(x,y):
  lines = [re.split(r'\s{2,}', line.strip()) for line in x.strip().split('\n')]
  df = pd.DataFrame(lines)
  ##df = pd.DataFrame(lines, columns=[[y,y,y,y],['Line#', 'Bearing#', 'Direction', 'Loads']])
  return df


In [7]:
##Extract table from txt file
##Main Code
text = textog
i=1
startp = "\n         -------------------------------------------------\n"
endp = "Auto generation details"
df_dict = {}

dframedc = pd.DataFrame()
dframell = pd.DataFrame()
dframebr = pd.DataFrame()
dframews = pd.DataFrame()
dframewl = pd.DataFrame()


loadnameindex = text.find("Loadcase ID:")
print(loadnameindex)
while loadnameindex != -1:
  if text[loadnameindex+13]!="W":
    name = text[loadnameindex+13:loadnameindex + 17]
  else:
    name = text[loadnameindex+13:loadnameindex + 45]
    name = name.replace("    Name: ", "-")
    name = name.replace("\n", "")
  #print(name)
  y=text.find(endp)

  #Convert to dframe
  data = text[text.find(startp)+len(startp):y]
  df = convSPtoDF(data,name)

  #Delete the dframe when there is 5th column
  if df.shape[1] >= 5:
    # Get the name of the 5th column (index 4, since counting starts from 0)
    col_to_delete = df.columns[4]

    # Drop the column
    df.drop(columns=[col_to_delete], inplace=True)

  #Add header
  df.columns = ['Line#', 'Bearing#', 'Direction', 'Loads-Kips']

  #Insert Column
  df.insert(0, name, [None] * len(df))


  #Add header with names
  #header_row = pd.DataFrame([[name,'','','']], columns=df.columns)
  #df = pd.concat([header_row, df], ignore_index=True)

  ##Put dframe into dictionary
  df_dict[name] = df

  ##print(df,"\n\n\n\n\n")
  ##dframe = pd.concat([dframe,df],axis=1)
  ##print(dframe)

  if "DC" in name:
    dframedc = pd.concat([dframedc,df],axis=1)
  elif "WS" in name:
    dframews = pd.concat([dframews,df],axis=1)
  elif "BR" in name:
    dframebr = pd.concat([dframebr,df],axis=1)
  elif "WL" in name:
    dframewl = pd.concat([dframewl,df],axis=1)
  else:
    dframell = pd.concat([dframell,df],axis=1)

  l=len(endp)
  text = text[y+l:]
  loadnameindex = text.find("Loadcase ID:")
  i=i+1

643


In [8]:
excelfilename = fn.replace(".txt",".xlsx")
with pd.ExcelWriter(excelfilename) as writer:
    dframedc.to_excel(writer, sheet_name='DC', index=False)
    dframell.to_excel(writer, sheet_name='LL', index=False)
    dframebr.to_excel(writer, sheet_name='BR', index=False)
    dframews.to_excel(writer, sheet_name='WS', index=False)
    dframewl.to_excel(writer, sheet_name='WL', index=False)

In [9]:
list(df_dict.keys())

['DC1 ',
 'LL1 ',
 'LL2 ',
 'LL3 ',
 'LL4 ',
 'LL5 ',
 'LL6 ',
 'LL7 ',
 'LL8 ',
 'LL9 ',
 'LL10',
 'LL11',
 'LL12',
 'LL13',
 'LL14',
 'LL15',
 'LL16',
 'LL17',
 'LL18',
 'LL19',
 'LL20',
 'LL21',
 'LL22',
 'LL23',
 'LL24',
 'LL25',
 'LL26',
 'LL27',
 'LL28',
 'LL29',
 'LL30',
 'LL31',
 'LL32',
 'LL33',
 'LL34',
 'LL35',
 'LL36',
 'LL37',
 'LL38',
 'LL39',
 'LL40',
 'LL41',
 'LL42',
 'LL43',
 'LL44',
 'LL45',
 'LL46',
 'LL47',
 'LL48',
 'LL49',
 'LL50',
 'LL51',
 'LL52',
 'LL53',
 'LL54',
 'LL55',
 'LL56',
 'LL57',
 'LL58',
 'BR1 ',
 'BR2 ',
 'BR3 ',
 'BR4 ',
 'BR5 ',
 'BR6 ',
 'BR7 ',
 'BR8 ',
 'BR9 ',
 'BR10',
 'BR11',
 'BR12',
 'BR13',
 'BR14',
 'BR15',
 'BR16',
 'BR17',
 'BR18',
 'BR19',
 'BR20',
 'BR21',
 'BR22',
 'BR23',
 'BR24',
 'BR25',
 'BR26',
 'BR27',
 'BR28',
 'BR29',
 'BR30',
 'BR31',
 'BR32',
 'BR33',
 'BR34',
 'BR35',
 'BR36',
 'BR37',
 'BR38',
 'BR39',
 'BR40',
 'BR41',
 'BR42',
 'BR43',
 'BR44',
 'BR45',
 'BR46',
 'BR47',
 'BR48',
 'BR49',
 'BR50',
 'BR51',
 'BR52',
 

In [ ]:
dframewl

,WL1 -Angle: 0 Mul,Line#,Bearing#,Direction,Loads-Kips
0,None,1,1,X,0.89
1,None,1,1,Y,1.18
2,None,1,1,Z,-0.00
3,None,1,2,X,0.89
4,None,1,2,Y,-0.00
5,None,1,2,Z,-0.00
6,None,1,3,X,0.89
7,None,1,3,Y,-0.00
8,None,1,3,Z,-0.00
9,None,1,4,X,0.89
